# Introduction

This is an implementation for Sequence to Sequence Encoder - Decoder Model using LSTM trained for the task of English-French translations. It is based on the paper - [Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215) and the notebook [Pytorch Seq2Seq](https://github.com/SethHWeidman/pytorch-seq2seq/blob/master/1%20-%20Sequence%20to%20Sequence%20Learning%20with%20Neural%20Networks.ipynb). 

It is implemented as an excerise to gain a deeper understanding of Encoder - Decoder models and build on it to explore advancements on the same.

Dataset used - [English - French Translations](https://www.kaggle.com/datasets/dhruvildave/en-fr-translation-dataset)

## Model training walkthrough

The idea behind such a model is to use the Encoder to encode the source sentence to a context vector, which the Decoder uses to generate the target sentence. 

Encoder Process - 
* Source sentence is split up into tokens and vocabularized. The sentences' tokens are passed into the model one by one using their indices from the vocab as input.
* The input is fed to an embedding layer which learns to generate an embedding vector (E) for the word. (A pre-trained embedding can be utilised here to improve accuracy and reduce training time by freezing the layer)
* E is fed into an RNN unit (either Vanilla RNN, LSTM or GRU) which also takes the hidden state of the previous token run as input (for the first token, it can be initialized as a randome vector or a tensor of 0s).
* E vector and the hidden state of the previous token pass (Hi-1) is used as input by the RNN unit to generate output and hidden state.
* The generated hidden state serves as the input to next token run, along with the next token's E vector.
* The context vector from the encoder is the output(hidden state) of the final token run from the source sentence.
* The context vector is used as the input to the decoder.

example pseudocode for one sentence pass through encoder - 

<code>
    vocab = build_vocab(source_sentence)
    sentence_tensor = [vocab[tokenize(word)] for word in source_sentence.split(' ')]
    hidden = 0
    for word_tensor in sentence_tensor:
        emb_vector = embed(word_tensor)
        hidden, output = rnn_unit(emb_vector, hidden) # hidden = output for 1 layer rnn
    context = hidden
</code>

<br/>
Decoder process - 
* Target sentence is split up into tokens and vocabularized. The sentences' tokens are passed into the model one by one using their indices from the vocab as input.
* The input is fed to an embedding layer which learns to generate an embedding vector (E) for the word. (A pre-trained embedding can be utilised here to improve accuracy and reduce training time by freezing the layer)
* E is fed into an RNN unit (either Vanilla RNN, LSTM or GRU) which also takes the hidden state of the previous token run as input (for the first token, it is the context vector returned from the encoder).
* The RNN unit uses E and hidden state from previous token as input and generates output vector and hidden state for the next token pass.
* The output vector is passed through a linear layer with output dimension as the vocab size. Argmax of linear layer output is taken to get the index of token generated in vocab.
* This token is then later used as input for the next token pass. 
* In case of teacher forcing approach, actual target token is used instead of generated token for next pass.

example pseudocode for one sentence pass through decoder - 

<code>
    vocab = build_vocab(target_sentence)
    sentence_tensor = [vocab[tokenize(word)] for word in target_sentence.split(' ')]
    hidden = context
    word_tensor = sentence_tensor[0]
    predicted_sentence = []
    for i in range(len(sentence_tensor)):
        emb_vector = embed(word_tensor)
        hidden, output = rnn_unit(emb_vector, hidden) # hidden = output for 1 layer rnn
        prediction = argmax(linear_layer(output))
        word_tensor = prediction
        predicted_sentence.append(prediction)
</code>
<br/>

### Module imports

In [1]:
import numpy as np
import pandas as pd
import spacy
import random
from torchtext.data.utils import get_tokenizer
import torch
import torchtext
from collections import Counter
from torchtext.vocab import vocab
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
torch.cuda.empty_cache()

import math
import time

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/en-fr-translation-dataset/en-fr.csv


In [2]:
SEED = 97

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

### Downloading Spacy models for use as tokenizers

In [3]:
!python -m spacy download en_core_web_sm
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 72.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 50.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


### Data import
Reading only 50000 rows for demonstration purposes. Training RNN is slow as it it takes input tokens one by one and I am limited by the compute available.

In [4]:
data = pd.read_csv('/kaggle/input/en-fr-translation-dataset/en-fr.csv', nrows=50000)
data = data.dropna().drop_duplicates()
data.head(5)

,en,fr
0,Changing Lives | Changing Society | How It Wor...,Il a transformé notre vie | Il a transformé la...
1,Site map,Plan du site
2,Feedback,Rétroaction
3,Credits,Crédits
4,Français,English


### Initializing the tokenizers

In [5]:
fr_tokenizer = get_tokenizer('spacy', language='fr_core_news_sm')
en_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')

### Splitting train, test and val datasets
Train - 85%, Val - 10%, Test - 5%

In [6]:
val_frac = 0.1
test_frac = 0.05

val_split_idx = int(len(data)*val_frac)
test_split_idx = int(len(data)*(val_frac + test_frac))

data_idx = list(range(len(data)))
np.random.shuffle(data_idx)

val_idx, test_idx, train_idx = data_idx[:val_split_idx], data_idx[val_split_idx:test_split_idx], data_idx[test_split_idx:]

df_train = data.iloc[train_idx].reset_index().drop('index',axis=1)
df_val = data.iloc[val_idx].reset_index().drop('index',axis=1)
df_test = data.iloc[test_idx].reset_index().drop('index',axis=1)

print('Length of train set - ', len(df_train))
print('Length of validation set - ', len(df_val))
print('Length of test set - ', len(df_test))

Length of train set -  42500
Length of validation set -  4999
Length of test set -  2500


In [7]:
def build_vocab(data, source_tokenizer, target_tokenizer):
    en_counter = Counter()
    fr_counter = Counter()
    translations = data.values.tolist()
    for translation in translations:
        en_counter.update(source_tokenizer(translation[0]))
        fr_counter.update(target_tokenizer(translation[1]))
    return vocab(en_counter, specials=['<unk>', '<pad>', '<bos>', '<eos>'], min_freq=2), vocab(fr_counter, specials=['<unk>', '<pad>', '<bos>', '<eos>'], min_freq=2)

### Building separate vocabs for English & French

In [8]:
en_vocab, fr_vocab = build_vocab(df_train, en_tokenizer, fr_tokenizer)
en_vocab.set_default_index(en_vocab['<unk>'])
fr_vocab.set_default_index(fr_vocab['<unk>'])

In [9]:
def data_process(data):
    translations = data.values.tolist()
    pairs = []
    for translation in translations:
        en_tensor = torch.tensor([en_vocab[token] for token in en_tokenizer(translation[0])][::-1],
                            dtype=torch.long)
        fr_tensor = torch.tensor([fr_vocab[token] for token in fr_tokenizer(translation[1])],
                            dtype=torch.long)
        pairs.append((en_tensor, fr_tensor))
    return pairs

In [10]:
train_data = data_process(df_train)
val_data = data_process(df_val)
test_data = data_process(df_test)

### Note:
Keeping a small batch size due to limitation of GPU size.

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 8
PAD_IDX = en_vocab['<pad>']
BOS_IDX = en_vocab['<bos>']
EOS_IDX = en_vocab['<eos>']

In [12]:
def generate_batch(data_batch):
    en_batch, fr_batch = [], []
    for (en_item, fr_item) in data_batch:
        en_batch.append(torch.cat([torch.tensor([BOS_IDX]), en_item, torch.tensor([EOS_IDX])], dim=0).to(device))
        fr_batch.append(torch.cat([torch.tensor([BOS_IDX]), fr_item, torch.tensor([EOS_IDX])], dim=0).to(device))  
        
    en_batch = pad_sequence(en_batch, padding_value=PAD_IDX)
    fr_batch = pad_sequence(fr_batch, padding_value=PAD_IDX)
    return en_batch, fr_batch

In [13]:
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)

### Encoder
* input_dim - source sentence vocab size
* emb_dim - output size of embedding layer
* hid_dim - Output size of hidden vector and output vector generated by RNN unit (LSTM in this case)
* n_layers - No. of RNN layers(RNN units) to stack or which the token passes through ( In case of multiple layers, the final context vector has final hidden vectors generated from each layer stacked on top of each other)
* dropout - percentage of droput to be used to avoid overfitting


In [14]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.lstm = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.lstm(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

### Decoder
* output_dim - target sentence vocab size
* emb_dim - output size of embedding layer
* hid_dim - Output size of hidden vector and output vector generated by RNN unit (LSTM in this case)
* n_layers - No. of RNN layers(RNN units) to stack or which the token passes through (in this example we keep the number of layers same in case of encoder and decoder, so the generated context vector can be directly used in decoder without the need of any summartion or passing through a linear layer for transformation)
* dropout - percentage of droput to be used to avoid overfitting


In [15]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.lstm = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

### Model - 
Contains the entire process explained earlier. As input it takes a batch of sentences and passes it through encoder to generate a context vector tensor for the entire batch. Then token by token it passes the sentence batch to decoder to generate output tokens.

In [16]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [17]:
len(en_vocab)

23165

In [18]:
len(fr_vocab)

25528

### Initializing the model

In [19]:
INPUT_DIM = len(en_vocab)
OUTPUT_DIM = len(fr_vocab)
ENC_EMB_DIM = 128
DEC_EMB_DIM = 128
HID_DIM = 256
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

### Initializing the model parameters
Distribution used is taken from the github notebook(replicated the approach mentioned in the paper)

In [20]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(23165, 128)
    (lstm): LSTM(128, 256, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(25528, 128)
    (lstm): LSTM(128, 256, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=256, out_features=25528, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [21]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 14,636,600 trainable parameters


### Initializing optimizer and loss functions

In [22]:
optimizer = optim.Adam(model.parameters())

TRG_PAD_IDX = fr_vocab['<pad>']
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

### Train method

In [23]:
def train(model, dataloader, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for _, (src, trg) in enumerate(dataloader):
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / (len(dataloader)*BATCH_SIZE)

### Evaluation method - no gradient calculation

In [24]:
def evaluate(model, dataloader, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for _, (src, trg) in enumerate(dataloader):

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / (len(dataloader)*BATCH_SIZE)

In [25]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [26]:
del df_train
del df_val
del df_test
del data
del en_tokenizer
del fr_tokenizer

### Training

In [27]:
MODEL_PATH = 'enc-dec-basic1-model.pt'

In [28]:
N_EPOCHS = 5
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_loader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, val_loader, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), MODEL_PATH)
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 13m 42s
	Train Loss: 0.779 | Train PPL:   2.179
	 Val. Loss: 0.778 |  Val. PPL:   2.177
Epoch: 02 | Time: 13m 36s
	Train Loss: 0.700 | Train PPL:   2.014
	 Val. Loss: 0.769 |  Val. PPL:   2.157
Epoch: 03 | Time: 13m 38s
	Train Loss: 0.663 | Train PPL:   1.941
	 Val. Loss: 0.761 |  Val. PPL:   2.140
Epoch: 04 | Time: 13m 42s
	Train Loss: 0.637 | Train PPL:   1.891
	 Val. Loss: 0.745 |  Val. PPL:   2.107
Epoch: 05 | Time: 13m 40s
	Train Loss: 0.617 | Train PPL:   1.854
	 Val. Loss: 0.738 |  Val. PPL:   2.092


In [29]:
model.load_state_dict(torch.load(MODEL_PATH))
test_loss = evaluate(model, test_loader, criterion)
print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 0.740 | Test PPL:   2.096 |
